In [25]:
import os

import numpy as np
from PIL import Image

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode

init_notebook_mode(connected=True)

## exclude debugging information 
import logging
logger = logging.getLogger()
logger.setLevel(level=logging.INFO)

#plotly.io.orca.config.executable = '/path/to/orca'


In [26]:
def evaluate(_enc_data: np.ndarray, _range_gt: np.ndarray) -> np.float:
    clf = RandomForestRegressor(max_depth= 5, n_estimators = 5, random_state=0)
    xy_range_gt = np.array([np.sin(_range_gt), np.cos(_range_gt)]).T
    
    N = _range_gt.shape[0]
    
    range_diff_all = np.zeros(0)
    range_predict_all = np.zeros(0)
    
    X = _enc_data
    y = xy_range_gt
    for i in range(10):
        ind_i = np.arange(np.int(i*N/10), np.int((i+1)*N/10))
        ind_i_rest = np.append(np.arange(0, np.int(i*N/10)), np.arange(np.int((i+1)*N/10), N))
        X_i = X[ind_i]
        X_i_rest = X[ind_i_rest]
        y_i = y[ind_i]
        y_i_rest = y[ind_i_rest]
        clf.fit(X_i_rest, y_i_rest)

        y_i_pred = clf.predict(X_i)

        range_predict = np.arctan2(y_i_pred[:,1], y_i_pred[:,0])

        range_gt = np.arctan2(y_i[:,1], y_i[:,0])

        range_diff = range_predict-range_gt

        range_diff = np.arctan2(np.sin(range_diff), np.cos(range_diff))

        range_diff_all = np.append(range_diff_all, range_diff)
        range_predict_all = np.append(range_predict_all, range_predict)
    
    return np.sqrt(np.sum(np.square(range_diff_all))/len(range_diff_all)), range_predict_all

def visualize(enc_data: np.ndarray, color_column):
    '''Used to visualize some of the embedding'''
    
    if enc_data.shape[1] == 2:
        scatter = go.Scatter3d(x=enc_data[:, 0], y=enc_data[:, 1], z=np.zeros(enc_data.shape[0]), mode='markers',
                               marker=dict(size=3, color=color_column, colorscale='viridis', opacity=1))
    elif enc_data.shape[1] == 3:
        size_column = 7 * np.ones(np.size(color_column))
        size_column[[0,31,51]] = 20
        scatter = go.Scatter3d(x=enc_data[:, 0], y=enc_data[:, 1], z=enc_data[:, 2], mode='markers',
                               marker=dict(size=size_column, color=color_column, colorscale='viridis', opacity=1))
    else:
        print("error: encoded data has neither 2 or 3 dimensions")
        return
    
    camera = dict(
        eye=dict(x=-2, y=-0.5, z=0.5)
    )

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        ),
        scene=dict(camera=camera),
    )
    
    fig = go.Figure(data=[scatter], layout=layout)
    iplot(fig)
    return fig

In [27]:

#### loading range ground truth ####

range_gt = np.load('data/range_ground_truth_train.npy')
print('range ground truth:', range_gt.shape)

#### loading images ####

TRAIN_DATA_PATH = "./data/imgs"

imgs_dir = TRAIN_DATA_PATH
imgs_names = sorted(os.listdir(imgs_dir))
imgs_count = len(imgs_names)

x = np.zeros((imgs_count, 304, 808, 3), dtype='uint8')
for idx in range(imgs_count):
    x[idx] = np.array(Image.open(imgs_dir + '/' + imgs_names[idx]))
print('images:', x.shape)

range ground truth: (1600,)
imgages: (1600, 304, 808, 3)


In [11]:

#### your encoding method (preprocessing, model inference, etc.) ####
def encode(x):
    return x


In [28]:

#### infer encoded data ####

x_enc = encode(x)

#### for benchmark load encrypted data #####
#x_enc = np.load('data/benchmark_enc_data_train.npy')


In [29]:

#### evaluate encoded data #####

std, range_predict = evaluate(x_enc, range_gt)
print("std:", std)

std: 0.19979829083541106


In [30]:

#### visualize encoded data #####

fig = visualize(x_enc, range_predict)

fig.write_image('enc_data.png')
